In [335]:
import torch as t
import math
import numpy as np
import matplotlib.pyplot as plt

# define problem:
def f(x):
    return t.tensor([[x[0]**2 + (x[1]-3)**2]])

def df(x):
    return t.tensor([[2*x[0],
                    2*(x[1]-3)]])

def g(x):
    return t.tensor([[x[1]**2-2*x[0],
                    (x[1]-1)**2+5*x[0]-15]])

def dg(x):
    return t.tensor([[-2,2*x[1]],
                    [5,2*(x[1]-1)]])

def solve_activeset(c,b,a,W):
    M_L = t.cat((W, a), 0)
    M_R = t.transpose(t.cat((a, t.zeros(a.size(dim=0),a.size(dim=0))), 1),0,1)
    M = t.cat((M_L, M_R), 1)
    U = t.transpose(t.cat((c, b), 1),0,1)
    sol = t.linalg.solve(M, U)
    print('sol:',sol)
    S = sol[[0,1]]
    MU = sol[2:]
    print('MU:',MU)
    return [S, MU]

def QP_solver(df0,g0,dg0):
    print('df0',df0)
    print('g0',g0)
    print('dg0',dg0)
    keepCal = True
    active = [] # [] for empty; [0] for g1; [1] for g2; [0,1] for g1 & g2
    i=0

    while keepCal:
        print('i=',i+1)
        mu0 = t.zeros(g0.size())
        g=g0[:,active]
        dg=dg0[active,:]
        print('g',g)
        print('dg',dg)

        [s, mu] = solve_activeset(-df0,-g,dg,H)
        print('s:',s)
        print('mu:',mu)
        mu0[:,active]=t.transpose(mu,0,1)
        print('mu0:',mu0)

        g_check = t.matmul(dg0,s)+t.transpose(g0,0,1)
        print('g_check:',g_check)
        mu_check = False

        if t.numel(mu)==0:
            mu_check = True
        elif t.min(mu)>0:
            mu_check = True
        else:
            remove=t.argmin(mu0).tolist()
            print('remove:',remove)
            active.remove(remove)
            print(active)

        if t.max(g_check) <= 0:
            if mu_check:
                keepCal = False
        else:
            add=t.argmax(g_check).tolist()
            print('add:',add)
            active.append(add)
            print(active)
        active = sorted(list(dict.fromkeys(active)))
        print(active)
    print(s)
    print(mu0)
    return [s, mu0]

# Initialization:
x0 = t.tensor([1.,1.])
x = x0

print('f:',f(x))
print('df:',df(x))
print('g:',g(x))
print('dg:',dg(x))

# Initialization of the Hessian matrix
H = t.eye(t.numel(x))

print('H:',H)

# Initialization of the Lagrange multipliers
LM_k = t.zeros(g(x).size())   # Start with zero Lagrange multiplier estimates
# Initialization of the weights in merit function
w = t.zeros(g(x).size())   # Start with zero weights

print('LM_k:',LM_k)
print('w:',w)

# Set the termination criterion
opt_eps = 1e-3
G_norm = t.norm(df(x)+t.matmul(LM_k,dg(x)))
print('G:',G_norm)

#while G_norm>opt_eps:

# QP
print('----------------------------------------')
[step_size, mu] = QP_solver(df(x),g(x),dg(x))
print('----------------------------------------')
print('step_size:',step_size)
print('mu:',mu)



f: tensor([[5.]])
df: tensor([[ 2., -4.]])
g: tensor([[ -1., -10.]])
dg: tensor([[-2.,  2.],
        [ 5.,  0.]])
H: tensor([[1., 0.],
        [0., 1.]])
LM_k: tensor([[0., 0.]])
w: tensor([[0., 0.]])
G: tensor(4.4721)
----------------------------------------
df0 tensor([[ 2., -4.]])
g0 tensor([[ -1., -10.]])
dg0 tensor([[-2.,  2.],
        [ 5.,  0.]])
i= 1
g tensor([], size=(1, 0))
dg tensor([], size=(0, 2))
sol: tensor([[-2.],
        [ 4.]])
MU: tensor([], size=(0, 1))
s: tensor([[-2.],
        [ 4.]])
mu: tensor([], size=(0, 1))
mu0: tensor([[0., 0.]])
g_check: tensor([[ 11.],
        [-20.]])
add: 0
[0]
[0]
i= 1
g tensor([[-1.]])
dg tensor([[-2.,  2.]])
sol: tensor([[0.7500],
        [1.2500],
        [1.3750]])
MU: tensor([[1.3750]])
s: tensor([[0.7500],
        [1.2500]])
mu: tensor([[1.3750]])
mu0: tensor([[1.3750, 0.0000]])
g_check: tensor([[ 0.0000],
        [-6.2500]])
[0]
tensor([[0.7500],
        [1.2500]])
tensor([[1.3750, 0.0000]])
--------------------------------------

In [301]:

import torch as t
x = t.tensor([1,1])
def g(x):
    return t.tensor([[x[1]**2-2*x[0]],
                    [(x[1]-1)**2+5*x[0]-15]])
print(t.zeros(g(x).size()))
print(10 > 9)
y = False
print(y)
test = [1,0,0,1,1]
test = list(dict.fromkeys(test))
print(sorted(test))

tensor([[0.],
        [0.]])
True
False
[0, 1]


In [1]:
# Verify
from scipy.optimize import minimize
fun = lambda x: x[0]**2 + (x[1]-3)**2

cons = ({'type': 'ineq', 'fun': lambda x:  2*x[0]-x[1]**2},
        {'type': 'ineq', 'fun': lambda x:  15-(x[1]-1)**2-5*x[0]})

x0 = [1,1]
res = minimize(fun, x0, method='SLSQP',
               constraints=cons)
res


     fun: 3.507468048250421
     jac: array([ 2.12041229, -3.08767256])
 message: 'Optimization terminated successfully'
    nfev: 17
     nit: 5
    njev: 5
  status: 0
 success: True
       x: array([1.06020715, 1.45616424])